#### 네이버 기사 제목 데이터 크롤링
- selenium 을 이용한 텍스트 데이터 수집
- thread 를 사용한 데이터 수집 (크롤링을 더 빠르게 하는 방법 : request와 response 중간의 ideal time도 다음 request하는 시간으로 사용 )
- 단점 : 끝나는 시점을 정확히 알 수 없음 (=비동기 방식 / 그냥 방식이 동기 방식)

In [2]:
from selenium import webdriver

In [7]:
page = 2
url = "https://news.naver.com/main/main.nhn?\
mode=LSD&mid=shm&sid1=105#&date=%2000:00:00&page={}".format(page)

In [8]:
driver = webdriver.Chrome()
driver.get(url)

In [16]:
articles = driver.find_elements_by_css_selector("#section_body li")
len(articles)

20

In [17]:
articles[0]

<selenium.webdriver.remote.webelement.WebElement (session="0c0efeae4de4cf440fc20e97db4a9f07", element="1f5422a0-b250-4966-aa6f-af5655d8b914")>

In [21]:
articles[0].find_element_by_css_selector("dt:not(.photo) > a").text

'[`디지털-뉴노멀`의 미래를 묻다] 5년후 10배 커지는 시장… 글로벌 선점경쟁'

In [22]:
article_list = []

for article in articles:
    title = article.find_element_by_css_selector("dt:not(.photo) > a").text
    article_list.append(title)

In [23]:
article_list

['[`디지털-뉴노멀`의 미래를 묻다] 5년후 10배 커지는 시장… 글로벌 선점경쟁',
 '美 통화감독청장 "디지털화폐 더는 무시할 수 없다"',
 "[김기자의 두잇(IT)] 작은귀에도 '빠지지 않는' 갤럭시 버즈 라이브",
 '미하엘 헤크마이어 머크 수석부사장 “디스플레이, 인간-데이터 잇는 핵심 인터페이스”',
 "[살까말까] 음식물 처리기, '음쓰' 걱정 날려줄까",
 '카카오페이지, 대만법인 설립…네이버 1위 아성에 도전',
 '마이데이터, 기존 업체 우선심사...득과 실은?',
 "애플 '아이폰12' LG 'Q92'…5G폰 선택지 넓어진다",
 '[달콤한 사이언스] 코로나 우습게 봐선 안되는 이유...재감염 가능성과 심각한 후유증 때문',
 '카카오, 역대 최대 규모 개발자 공채…세자릿수 뽑는다(종합)',
 '정부-의료계, 대화 물꼬 텄지만…의료 공백 우려는 `여전`',
 '굿바이, 스물다섯…인터넷익스플로러',
 '당신의 개와 고양이, 신종 코로나에 무사한가요?',
 "블랙베리 이어 노키아도 '꿈틀'...인도 저가폰 시장 공략",
 'IT업계, 엔터테인먼트 영토확장 불 붙었다',
 "정부 주도 'K-OTT 협의체' 가동…'공룡' 넷플릭스 대항마 나올까",
 "'서머너즈 워'에 '스트리트 파이터' 접목…게임도 콜라보 시대",
 '"4.5조원 日 웹툰 잡아라"…카카오·네이버, 시장공략 가속',
 '5G 스마트폰 시장 출사표 던진 구글, 이번에는 다를까',
 '뉴스 개인브랜딩 강화하는 네이버… 기자명 클릭수 2배로']

In [24]:
driver.quit()

In [25]:
# 함수로 만들기
article_list = []

def get_articles(page):
    driver = webdriver.Chrome()
    url = "https://news.naver.com/main/main.nhn?\
mode=LSD&mid=shm&sid1=105#&date=%2000:00:00&page={}".format(page)
    driver.get(url)
    articles = driver.find_elements_by_css_selector("#section_body li")
    for article in articles:
        title = article.find_element_by_css_selector("dt:not(.photo) > a").text
        article_list.append(title)
    print("crawling : {}".format(page))
    driver.quit()

In [26]:
%%time
for page in range(1, 5):
    get_articles(page)

crawling : 1
crawling : 2
crawling : 3
crawling : 4
CPU times: user 202 ms, sys: 45.7 ms, total: 248 ms
Wall time: 18.8 s


In [28]:
len(article_list) # 20개씩 4페이지이니까

80

#### thread 사용

In [29]:
import threading

In [33]:
article_list = []

for page in range(1, 5):
    th = threading.Thread(target=get_articles, args=(page,))  # page 뒤에 , 를 붙이는 이유는 
                                             # 연산자 우선순위 ()가 아닌 튜플로 선언해 주기 위해서
    #target에 실행시킬 함수를 넣는 것
    th.start()

In [34]:
import time
sec = 0
while len(article_list) < 80:
    time.sleep(1)
    sec += 1
    print(sec, end=" ")
    
# 위에꺼랑 동시에 실행시키면 초가 보이겠죠

1 2 3 4 5 6 7 crawling : 2
8 crawling : 3
9 10 crawling : 1
crawling : 4
11 